In [1]:
# Import library
import requests
import pandas as pd
import numpy as np
import json
from tqdm import tqdm
import base64
from requests.exceptions import ReadTimeout
import csv
import datetime

In [2]:
# IDs Andrea
client_id = "14173c8287014a8088fc75444dff7460"
client_secret = "58ff1fa48e1f4c9388c6c6fa217c521d" # non funzionano più

# IDs Donato
# client_id = 'edad319501224e3eb0f30b3ac7e768f3'
# client_secret = '91bb67d75fa448559f4555dc51123614' # non funzionano più

# IDs Andrea 2
# client_id = '4ef878db53474c3a8363950706680c6a'
# client_secret = '28f093e6d9ba4009b248f17c7bace0c9' # non funzionano più

# IDs Donato 2
# client_id = "14173c8287014a8088fc75444dff7460"
# client_secret = "58ff1fa48e1f4c9388c6c6fa217c521d" # non funzionano più

# IDs Donato 3
# client_id = "50e6bc9c836c408985d94b7132a3d431"
# client_secret = "9d13891d708e4a3590595f3174b099df" # non funzionano più

# IDs Donato 4
# client_id = "3dc41469661f45289ed6c77f78e27ea9"
# client_secret = "285603eb2fa243709339a2c2ed5aaa3a"

# IDs Donato 5
#client_id = "8f7b49098bda43c68b8413fa18d71d32"
#client_secret = "9c67e9e2a0a5447db1445a38646711f2"

In [3]:
headers = {}
client_creds = f"{client_id}:{client_secret}"
client_creds_base64 = base64.b64encode(client_creds.encode())
token_url = 'https://accounts.spotify.com/api/token'
token_data = {"grant_type": "client_credentials"}
token_headers = {"Authorization":f"Basic {client_creds_base64.decode()}" }

def get_new_token():
    r = requests.post(token_url, headers=token_headers, data=token_data)
    valid_request = r.status_code in range(200, 299)

    if valid_request:
        r = r.json()
        access_token = r['access_token']
        expires_in = r["expires_in"]    # in second
        # print(access_token, expires_in)

        now = datetime.datetime.now()
        expires = now + datetime.timedelta(seconds=expires_in)
        # did_expire = expires < now

        # Construct Spotify API query
        headers = {
            'Authorization': 'Bearer '+ access_token,
            'Content-type': 'application/json',
        }
    return expires, headers

expires, headers = get_new_token()

BQCWfU8agm3lEMILFv9Xb6Z3VKicZO4RNLl6hI7FmZuJLlWYdM8tW45jPjANOYeRXuIBBawiJ4Yj_PNaVJ4 3600


In [ ]:
track_url = 'https://api.spotify.com/v1/recommendations/available-genre-seeds'

# Query Spotify API
try:
    if expires < datetime.datetime.now(): 
        print("Get new Token")
        expires, headers = get_new_token()
    genres = requests.get(track_url, headers=headers).json()
    # print(json.dumps(genres, indent=4))
except Exception as e: print("\n", e)


# Create a json file for genres list
with open("genres.json", 'w') as f:
   json.dump(genres, f, indent=4)

In [ ]:
min_number_of_tracks = 1000
max_null_iteration = 10
stored_tracks = {} # {track_id: genre}
old_tot_track = 0
for genre in tqdm(genres["genres"]):
    number_of_tracks = 0
    null_iteration_count = 0

    track_url = 'https://api.spotify.com/v1/recommendations?' + 'seed_genres=' + genre + "&limit=100"

    while number_of_tracks < min_number_of_tracks and null_iteration_count < max_null_iteration:
        try:
            if expires < datetime.datetime.now(): 
                print("Get new Token")
                expires, headers = get_new_token()
            # tracks_list = sp.recommendations(seed_genres=[genre], limit=100)
            tracks_list = requests.get(track_url, headers=headers).json()                 
        except Exception as e: print("\n", e)
        
        null_iteration = True
        
        if "tracks" in tracks_list:
            for i, track in enumerate(tracks_list["tracks"]):
                if track["id"] not in stored_tracks:
                    stored_tracks[track["id"]] = genre
                    number_of_tracks += 1
                    null_iteration = False

        if null_iteration:
            null_iteration_count += 1
            
    #print(genre, len(stored_tracks)-old_tot_track)
    #old_tot_track = len(stored_tracks)
    # Create a json file for genres list
    with open("stored_tracks.json", 'w') as f:
       json.dump(stored_tracks, f, indent=4)
    
        

In [ ]:
# Create a json file for genres list
with open("stored_tracks.json", 'w') as f:
   json.dump(stored_tracks, f, indent=4)

In [7]:
columns = ["id", "track_name", "track_explicit",  "track_popularity", "album_name", "album_release_date", "album_release_date_precision",
           "artist_name", "audio_avg_pitches", "audio_avg_timbre",
           "audio_acousticness", "audio_danceability", "audio_duration_ms", "audio_energy", "audio_instrumentalness",
           "audio_key_1", "audio_liveness", "audio_loudness", "audio_mode_1", "audio_speechiness", "audio_tempo",
           "audio_time_signature", "audio_valence", "track_uri", "track_genre"]

with open('dataframe.csv', 'w', newline='') as f:
    write = csv.writer(f)
    write.writerow(columns)
    #write.writerows(list_of_track_audio_features)
    f.close()

In [8]:
list_of_track_audio_features = []

with open('stored_tracks.json') as f:
    stored_tracks = json.load(f)

for i,track in enumerate(tqdm(stored_tracks)):

    if i > 14124:
      break
    if expires < datetime.datetime.now(): 
        print("Get new Token")
        expires, headers = get_new_token()
   
            
    try:
      track_info_url = "https://api.spotify.com/v1/tracks/" + track
      track_info = requests.get(track_info_url, headers=headers)
      track_audio_analysis_url = "https://api.spotify.com/v1/audio-analysis/" + track
      track_audio_analysis = requests.get(track_audio_analysis_url, headers=headers)
      features_url = "https://api.spotify.com/v1/audio-features/" + track
      features = requests.get(features_url, headers=headers)  

      valid_request_1 = track_info.status_code in range(200, 299)
      valid_request_2 = track_audio_analysis.status_code in range(200, 299)
      valid_request_3 = features.status_code in range(200, 299)

      if (valid_request_1 and valid_request_2 and valid_request_3):
        track_info = track_info.json()
        track_audio_analysis = track_audio_analysis.json()
        features = features.json()
      else:
        print("\n STATUS:", track_info.status_code, track_audio_analysis.status_code, features.status_code)
        continue
    except Exception as e:
      print(e)
      continue

    track_name = track_info["name"]     
    track_explicit = track_info['explicit']
    track_popularity = track_info["popularity"]
    album_info = track_info['album']
    album_name = album_info['name']
    album_release_date = album_info['release_date']
    album_release_date_precision = album_info['release_date_precision']
    artist_info = track_info['artists'][0]
    artist_name = artist_info['name']
    track_info = [track_name, track_explicit, track_popularity,
                  album_name, album_release_date, album_release_date_precision,
                  artist_name]
  
    #Add in some Spotify Audio Analysis Vectors
    num_seg = 0
    pitches = np.zeros(12)
    timbre = np.zeros(12)   
    if "segments" in track_audio_analysis:
        for _, j in enumerate(track_audio_analysis['segments']):
            pitches += np.array(j['pitches'])
            timbre += np.array(j['timbre'])
            num_seg+=1
    track_features_avg_pitches = list(pitches/num_seg)
    track_features_avg_timbre = list(timbre/num_seg)
    track_audio_analysis_list = [track_features_avg_pitches, track_features_avg_timbre]
  
    # Get audio features for this specific track
    features_list = [features['acousticness'], features['danceability'], features['duration_ms'],
                      features['energy'], features['instrumentalness'], features['key'], 
                      features['liveness'], features['loudness'], features['mode'],  
                      features['speechiness'], features['tempo'], features['time_signature'],
                      features['valence'], features['uri']]
    
    trackData = [track] + track_info + track_audio_analysis_list + features_list + [stored_tracks[track]]
    list_of_track_audio_features.append(trackData)

    with open('dataframe.csv', 'a+', newline='') as f:
    # using csv.writer method from CSV package
        write = csv.writer(f)
        write.writerow(trackData)
        f.close()

  2%|▏         | 1242/65214 [08:36<7:25:03,  2.40it/s]


 STATUS: 500 200 200


  2%|▏         | 1333/65214 [09:20<9:31:21,  1.86it/s]


 STATUS: 500 200 200


  2%|▏         | 1585/65214 [11:19<8:42:46,  2.03it/s]

...


 STATUS: 500 200 200


  3%|▎         | 2241/65214 [15:58<14:44:50,  1.19it/s]


 STATUS: 500 200 200


  7%|▋         | 4487/65214 [32:26<7:18:57,  2.31it/s]


KeyboardInterrupt: ignored

In [ ]:
spotify_track = pd.DataFrame(list_of_track_audio_features, columns = columns)
spotify_track.head()